<a href="https://colab.research.google.com/github/trisha025/Image_Classification_TransferLearning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch
!pip3 install torchvision

In [0]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [0]:
image_transforms = {
    'train' : transforms.Compose([
                                  transforms.RandomResizedCrop(size=256, scale=(0.8,1.0)),
                                  transforms.RandomRotation(degrees=15),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])
    ]),
    'valid' : transforms.Compose([
                                  transforms.Resize(size=256),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])
    ]),
    'test' : transforms.Compose([
                                 transforms.Resize(size=256),
                                 transforms.CenterCrop(size=224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])
    ])
}

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_directory = '/content/gdrive/My Drive/caltech/train'
valid_directory = '/content/gdrive/My Drive/caltech/valid'
test_directory = '/content/gdrive/My Drive/caltech/test'

In [0]:
bs = 32
num_classes = 10

In [0]:
data = {
    'train' : datasets.ImageFolder(root=train_directory,
                                   transform=image_transforms['train']),
        'valid' : datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
        'test' : datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
}

In [16]:
train_data_size = len(data['train'])
print(train_data_size)

600


In [18]:
valid_data_size = len(data['valid'])
print(valid_data_size)

100


In [19]:
test_data_size = len(data['test'])
print(test_data_size)

409


In [0]:
train_data = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data = DataLoader(data['test'], batch_size=bs, shuffle=True)